# 캘리포니아 주택 가격 데이터 이용한 `ElasticNet()` 모델 성능 높이기

## ElasticNet() 모델 성능 평가

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split

california = fetch_california_housing()

model = ElasticNet(alpha=0.1, l1_ratio=0.2)

X_train, X_test, y_train, y_test = train_test_split(california.data, california.target, test_size=0.2)

model.fit(X_train, y_train)

print("학습 데이터 점수: {}".format(model.score(X_train, y_train)))
print("평가 데이터 점수: {}".format(model.score(X_test, y_test)))

학습 데이터 점수: 0.5855823837201637
평가 데이터 점수: 0.5913182684639027


## GridSearchCV() 통한 최적의 하이퍼 파라미터 찾기

In [ ]:
import pandas as pd
from sklearn.model_selection import GridSearchCV

param_grid = [ {'alpha': [0.001, 0.01, 0.05, 0.1, 0.2, 0.3],
                'l1_ratio': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6] } ]

gs = GridSearchCV(estimator=ElasticNet(), param_grid=param_grid, cv=5)
result = gs.fit(california.data, california.target)

print("최적 점수: {}".format(result.best_score_))
print("최적 파라미터: {}".format(result.best_params_))
print(gs.best_estimator_)
pd.DataFrame(result.cv_results_)

최적 점수: 0.5531766524937997
최적 파라미터: {'alpha': 0.001, 'l1_ratio': 0.1}
ElasticNet(alpha=0.001, copy_X=True, fit_intercept=True, l1_ratio=0.1,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=None, selection='cyclic', tol=0.0001, warm_start=False)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_l1_ratio,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.029199,0.007706,0.001256,0.000075,0.001,0.1,"{'alpha': 0.001, 'l1_ratio': 0.1}",0.550659,0.467596,0.550788,0.536042,0.660798,0.553177,0.061987,1
1,0.021900,0.002376,0.001138,0.000008,0.001,0.2,"{'alpha': 0.001, 'l1_ratio': 0.2}",0.550743,0.467529,0.550794,0.535931,0.660827,0.553165,0.062021,2
2,0.021148,0.002339,0.001145,0.000019,0.001,0.3,"{'alpha': 0.001, 'l1_ratio': 0.3}",0.550828,0.467461,0.550799,0.535820,0.660855,0.553153,0.062055,3
3,0.021474,0.001727,0.001155,0.000008,0.001,0.4,"{'alpha': 0.001, 'l1_ratio': 0.4}",0.550913,0.467393,0.550805,0.535708,0.660884,0.553140,0.062090,4
4,0.027040,0.014495,0.001161,0.000021,0.001,0.5,"{'alpha': 0.001, 'l1_ratio': 0.5}",0.550998,0.467324,0.550810,0.535595,0.660912,0.553128,0.062124,5
5,0.020678,0.002369,0.001142,0.000011,0.001,0.6,"{'alpha': 0.001, 'l1_ratio': 0.6}",0.551084,0.467255,0.550815,0.535481,0.660941,0.553115,0.062159,6
6,0.017311,0.001369,0.001171,0.000077,0.01,0.1,"{'alpha': 0.01, 'l1_ratio': 0.1}",0.561303,0.462766,0.550248,0.528318,0.661939,0.552915,0.064336,7
7,0.016826,0.001808,0.001130,0.000010,0.01,0.2,"{'alpha': 0.01, 'l1_ratio': 0.2}",0.561839,0.461978,0.550146,0.527041,0.662078,0.552616,0.064718,8
8,0.016210,0.001448,0.001150,0.000023,0.01,0.3,"{'alpha': 0.01, 'l1_ratio': 0.3}",0.562366,0.461136,0.550013,0.525691,0.662203,0.552282,0.065121,9
9,0.016415,0.001890,0.001146,0.000013,0.01,0.4,"{'alpha': 0.01, 'l1_ratio': 0.4}",0.562877,0.460234,0.549847,0.524262,0.662313,0.551907,0.065547,10


## Preprocessing - `StandardScaler`: 표준화

In [ ]:
from sklearn.preprocessing import StandardScaler

california = fetch_california_housing()
california_df = pd.DataFrame(data=california.data, columns=california.feature_names)
california_df.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,3.870671,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704
std,1.899822,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000
25%,2.563400,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000
50%,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000
75%,4.743250,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000
max,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000


In [ ]:
scaler = StandardScaler()
california_scaled = scaler.fit_transform(california_df)
california_df_scaled = pd.DataFrame(data=california_scaled, columns=california.feature_names)
california_df_scaled.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
count,2.064000e+04,2.064000e+04,2.064000e+04,2.064000e+04,2.064000e+04,2.064000e+04,2.064000e+04,2.064000e+04
mean,3.734255e-16,8.557001e-16,2.704111e-16,-1.531384e-16,-6.465442e-17,6.064808e-19,1.256263e-15,-6.527810e-15
std,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00
min,-1.774299e+00,-2.196180e+00,-1.852319e+00,-1.610768e+00,-1.256123e+00,-2.290000e-01,-1.447568e+00,-2.385992e+00
25%,-6.881186e-01,-8.453931e-01,-3.994496e-01,-1.911716e-01,-5.638089e-01,-6.171062e-02,-7.967887e-01,-1.113209e+00
50%,-1.767951e-01,2.864572e-02,-8.078489e-02,-1.010650e-01,-2.291318e-01,-2.431585e-02,-6.422871e-01,5.389137e-01
75%,4.593063e-01,6.643103e-01,2.519615e-01,6.015869e-03,2.644949e-01,2.037453e-02,9.729566e-01,7.784964e-01
max,5.858286e+00,1.856182e+00,5.516324e+01,6.957171e+01,3.025033e+01,1.194191e+02,2.958068e+00,2.625280e+00


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(california_df_scaled, california.target, test_size=0.2)
model = ElasticNet(alpha=0.001, l1_ratio=0.1)
model.fit(X_train, y_train)

print("훈련 데이터 점수: {}".format(model.score(X_train, y_train)))
print("평가 데이터 점수: {}".format(model.score(X_test, y_test)))

훈련 데이터 점수: 0.603019978699828
평가 데이터 점수: 0.6188119380297581


## cross_val_score(): 교차 검증

In [ ]:
import numpy as np
from sklearn.model_selection import cross_val_score, cross_validate

scores = cross_val_score(model, california.data, california.target, cv=5)

print("교차 검증 정확도: {}".format(scores))
print("교차 검증 정확도: {} +/- {}".format(np.mean(scores), np.std(scores)))

교차 검증 정확도: [0.55065917 0.46759593 0.5507877  0.53604231 0.66079815]
교차 검증 정확도: 0.5531766524937997 +/- 0.06198721845014342
